## ROME TO FAP LINKING TABLE PROCESSING

### Purpose
This script processes the ROME to FAP linking table, structuring occupational classification codes for analysis.

### Steps
1. **Load Data:** Import the Excel file and select the relevant sheet.
2. **Extract Categories:** Assign hierarchical FAP codes (`fap22`, `fap87`, `fap225`).
3. **Handle Missing Data:** Remove empty rows and forward-fill missing values.
4. **Assign Professional Families:** Map `famille_pro` names based on category levels.
5. **Clean & Filter:** Keep relevant rows, drop unnecessary columns, and reorder data.

### Output
A structured dataset with FAP codes, professional families, PCS categories, and ROME classifications for streamlined analysis and merging.


### 1- LIBRARIES AND PATHS

In [5]:
# IMPORT LIBRARIES
from pathlib import Path # To set relative paths
import pandas as pd

In [14]:
# GETTING PROJECT'S ROOT DIRECTORY
base_folder = Path().resolve()  # CURRENT WORKING DIRECTORY
main_folder = base_folder.parent

# SETTING PATHS AND DIRECTORIES
fap_to_rome = main_folder / "data" / "linking tables" / "Rome-V3 vers Fap-2009.xls"
output_path = main_folder / "data" / "linking tables" / "Rome_to_Fap_processed.csv"

### 2- WORKFLOW

In [19]:
# IMPORT ROME TO FAP LINKING TABLE
link = pd.read_excel(fap_to_rome, sheet_name=1)

# ASSIGNING CATEGORY (FAP COLUMN)
category_col = link.columns[0]

# FUNCTION TO EXTRACT CATEGORIES IN SEPARATE COLUMNS
def extract_categories(code):
    if pd.isna(code):
        return pd.Series([None, None, None])  # Handle missing values
    elif len(code) == 1:  # Level 1 (e.g., 'A')
        return pd.Series([code, None, None])
    elif len(code) == 3:  # Level 2 (e.g., 'A0Z')
        return pd.Series([code[0], code, None])
    else:  # Level 3 (e.g., 'A0Z00')
        return pd.Series([code[0], code[:3], code])

# APPLY FUNCTION TO CREATE NEW CATEGORY COLUMN
link[['fap22', 'fap87', 'fap225']] = link[category_col].apply(extract_categories)

# DROP BLANK ROWS
link = link.dropna(subset=['fap22', 'fap87', 'fap225', "ROME"], how='all')

# CREATE COLUMNS FOR PROFESSIONAL FAMILY FOR EACH FAP CATEGORY
link["famille_pro22"] = link["Familles professionnelles"].where(link["fap22"].notna() & link["fap87"].isna() & link["fap225"].isna())
link["famille_pro87"] = link["Familles professionnelles"].where(link["fap22"].notna() & link["fap87"].notna() & link["fap225"].isna())
link["famille_pro225"] = link["Familles professionnelles"].where(link["fap22"].notna() & link["fap87"].notna() & link["fap225"].notna())

# REPLACE NA WITH CLOSEST VALUE UPWARD
link = link.fillna(method="ffill")

# KEEP COLUMNS AND DATA OF INTEREST
link = link[(link["FAP"].str.len() == 5)]
link = link.drop(columns=["FAP", "Familles professionnelles", "Qualification"])
link = link[["fap22", "famille_pro22", "fap87", "famille_pro87", "fap225", "famille_pro225", "PCS",
        "Professions et catégories socioprofessionnelles", "ROME", "Répertoire Opérationnel des Métiers et des Emplois"]]

# PRINT THE TABLE
link.head(-1)

,fap22,famille_pro22,fap87,famille_pro87,fap225,famille_pro225,PCS,Professions et catégories socioprofessionnelles,ROME,Répertoire Opérationnel des Métiers et des Emplois
8,A,"Agriculture, marine, pêche",A0Z,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A0Z00,Agriculteurs indépendants,111a,Agriculteurs sur petite exploitation de céréal...,NaN,NaN
16,A,"Agriculture, marine, pêche",A0Z,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A0Z01,Éleveurs indépendants,111d,"Éleveurs d'herbivores, sur petite exploitation",NaN,NaN
23,A,"Agriculture, marine, pêche",A0Z,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A0Z02,"Bûcherons, sylviculteurs indépendants",122b,"Exploitants forestiers indépendants, de 0 à 9 ...",NaN,NaN
25,A,"Agriculture, marine, pêche",A0Z,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A0Z40,Agriculteurs salariés,691e,Ouvriers agricoles sans spécialisation particu...,A1416,"Polyculture, élevage"
27,A,"Agriculture, marine, pêche",A0Z,"Agriculteurs, éleveurs, sylviculteurs, bûcherons",A0Z41,Éleveurs salariés,691b,Ouvriers de l'élevage,A1403,Aide d'élevage agricole et aquacole
...,...,...,...,...,...,...,...,...,...,...
1405,W,"Enseignement, formation",W0Z,Enseignants,W0Z91,Directeurs d'établissement scolaire et inspect...,227b,Indépendants gestionnaires d'établissements pr...,K2103,Direction d'établissement et d'enseignement
1410,W,"Enseignement, formation",W0Z,Enseignants,W0Z92,Professeurs du supérieur**,342a,Enseignants de l'enseignement supérieur,K2108,Enseignement supérieur
1416,W,"Enseignement, formation",W1Z,Formateurs,W1Z80,Formateurs,423a,Moniteurs d'école de conduite,K2110,Formation en conduite de véhicules
1417,W,"Enseignement, formation",W1Z,Formateurs,W1Z80,Formateurs,423b,Formateurs et animateurs de formation continue,K2111,Formation professionnelle


### 3- EXPORT FILE

In [17]:
# EXPORT DATASET 
link.to_csv(output_path, index=False)